In [1]:
import os
current_dir = %pwd
print(current_dir)
os.chdir("../")
changed_dir = %pwd
print(changed_dir)

c:\Users\subhash_dixit\Desktop\Learning\projects\end_to_end_text_summarization_project\research
c:\Users\subhash_dixit\Desktop\Learning\projects\end_to_end_text_summarization_project


### entity

In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    

### configuration manager

In [3]:
from text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
    

### Components

In [5]:
import os
import urllib.request as request
import zipfile
from text_summarizer.logging import logger
from text_summarizer.utils.common import get_size

In [6]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

### Pipeline

In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-09-02 02:16:03,906: INFO: common: YAML file: config\config.yaml loaded successfully]
[2024-09-02 02:16:03,911: INFO: common: YAML file: params.yaml loaded successfully]
[2024-09-02 02:16:03,913: INFO: common: Created directory at: artifacts]


[2024-09-02 02:16:03,916: INFO: common: Created directory at: artifacts/data_ingestion]
[2024-09-02 02:16:08,097: INFO: 1434958058: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: BBC6:106B51:10BE1D:16559C:66D4D28C
Accept-Ranges: bytes
Date: Sun, 01 Sep 2024 20:46:05 GMT
Via: 1.1 varnish
X-Served-By: cache-del21724-DEL
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1725223565.953395,VS0,VE689
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 313cb78ec25895d38c535f84bb97e05f7527807d
Expires: Su